# Asteroid Ivezić JPL Horizon vs OpenOrb comparison

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: I11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:
* table color-coding: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

# Display-ready Zone

## Set-up

In [1]:
%matplotlib inline

import sys
import re
import numpy as np
import pandas as pd
import jinja2
import astropy as ap
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import QTable
from astropy.io import ascii
import matplotlib.pyplot as plt
import astroquery as aq
from astroquery.jplhorizons import Horizons
import pyoorb as oo
oo.pyoorb.oorb_init()

0

In [2]:
# "Global" variables: times (UTC), location, object (find a way to generalize objects)
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:01'
internal_start = ap.time.Time(start_time)
internal_stop = ap.time.Time(stop_time)
element_time = internal_start.jd
Gemini_S = 'I11@399'
obj_id = 'Ivezic'

## JPL Horizons

In [3]:
def get_ephem_jpl(obj_id, element_time):
    el_obj = Horizons(id=obj_id, location= '500@10',
               epochs=element_time)
    el_jpl = el_obj.elements()
    ephem_obj = Horizons(id= obj_id, location= Gemini_S,
               epochs={'start': start_time, 'stop':stop_time,
                      'step':'1d'})
    ephem_jpl = ephem_obj.ephemerides()
    
    RA_jpl = ephem_jpl['RA']
    DEC_jpl = ephem_jpl['DEC']
    coord_jpl = np.array([RA_jpl, DEC_jpl]) * u.deg
    
    return el_jpl, coord_jpl

## OpenOrb/ pyoorb

In [4]:
# function to reorganize JPL Horizons output into pyoorb-acceptable inputs. Expand for multiple orbits next
def reorganizer(orbits, epoch):
    '''
    Parameters
    ------
    orbits : `~numpy.ndarray` (N, 18)
    epoch : `~numpy.ndarray` (3652, 2)
        Constrained to cometary format.
    Returns
    -------
    new_array : `~numpy.ndarray` (N, 12)
        Orbits formatted in the format expected by PYOORB. 
            orbit_id : index of input orbits
            elements x6: orbital elements of propagated orbits
            orbit_type : orbit type
            epoch_mjd : epoch of the propagate orbit
            time_scale : time scale of output epochs
            H/M1 : absolute magnitude
            G/K1 : photometric slope parameter
    '''
    
    temp = orbits.copy()
    temp = temp.as_array().data
    if temp.shape == (6,):
        num_orbits = 1
    else:
        num_orbits = temp.shape[0]
        
    for i in range(num_orbits):
        ids = i
        orbit_type = 2
        time_scale = 3
        
    # elements x6
    q = temp[0][6]
    e = temp[0][5]
    incl = np.deg2rad(temp[0][7])
    longnode = np.deg2rad(temp[0][8])
    argper = np.deg2rad(temp[0][9])
    peri_epoch = ap.time.Time(temp[0][10], format='jd').mjd

    mag = temp[0][3]
    slope = temp[0][4]
    
    if num_orbits > 1:
        new_array = np.array(
            np.array([
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]), 
            dtype=np.double, 
            order='F')
    else:
        new_array = np.array(
            [[
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]], 
            dtype=np.double,
            order='F')
    
    return new_array

In [5]:
def get_ephem_OpenOrb(el_jpl):
    # time conversions, epochs for pyoorb to work
    element_time_pyoorb = internal_start.mjd
    start_pyoorb = internal_start.mjd
    stop_pyoorb = internal_stop.mjd
    peri_time = ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd
    
    #conversion and implementation
    pyoorb_formatted = reorganizer(el_jpl, start_pyoorb)
    t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
    mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
    epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
    ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=pyoorb_formatted,
                                         in_obscode='I11',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')
    
    RA_OpenOrb = ephem_pyoorb[0][:,1]
    DEC_OpenOrb = ephem_pyoorb[0][:,2]
    coord_OpenOrb = np.array([RA_OpenOrb,DEC_OpenOrb]) * u.deg
    
    return coord_OpenOrb

## OrbFit

Use these:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_fwf.html
https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#csv-text-files
sign in [50) thing


In [7]:
def get_ephem_OrbFit():
    ! (cd 2019OK && ./fitobs.x < ast.inp > /dev/null)
    df = pd.read_fwf('2019OK/202930.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55), (56, 61)])
    
    df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
    df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
    df.loc[df[4] == '-', "DEC"] *= -1
    
    RA_OrbFit = df["RA"]
    DEC_OrbFit = df["DEC"]
    coord_OrbFit = np.array([RA_OrbFit, DEC_OrbFit]) * u.deg
    return coord_OrbFit

## Outputs and comparison

In [8]:
# main function. Will take in object id's and convert&conquer to all integrators
def get_ephems(obj_id):
    el_jpl, coord_jpl = get_ephem_jpl(obj_id, element_time)
    coord_OpenOrb = get_ephem_OpenOrb(el_jpl)
    coord_OrbFit = get_ephem_OrbFit()
    return el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit

In [9]:
el_jpl, coord_jpl, coord_OpenOrb, coord_OrbFit = get_ephems(obj_id)

## Difference metric

In [10]:
def max_diff(param1, param2):
    '''
    Parameters
    ----------
    param1: parameters from 1st integrator, `numpy array`
    param2: parameters from 2nd integrator, `numpy array` 
    
    Returns
    -------
    diff_matrix: separation between ephem1 and ephem2, `numpy array`
    '''
    
    #coordinates
    coord1 = SkyCoord(param1[0], param1[1], frame='icrs', unit="deg")
    coord2 = SkyCoord(param2[0], param2[1], frame='icrs', unit="deg")
    sep = coord1.separation(coord2)
    
    # magnitudes
    #mag1 = ephem1['H']
    #mag2 = ephem2['H']
    
    #magnitude = mag2 - mag1
    
    # matrix
    # degs, mins, secs; extract secs
    diff_matrix = sep.arcsec
        
    return diff_matrix.flatten()

In [11]:
great_circle_diff1 = max_diff(coord_jpl, coord_OpenOrb)
print("Precision of OpenOrb w/r/t JPL:", great_circle_diff1)

print()
great_circle_diff2 = max_diff(coord_jpl, coord_OrbFit)
print("Precision of Orbfit w/r/t  JPL:", great_circle_diff2)

#HALF A PIXEL (1p is 0.2 arcs (but also photometry can improve measurements))

Precision of OpenOrb w/r/t JPL: [0.00785716 0.01113825 0.00619841 ... 0.1361494  0.12737095 0.1156872 ]

Precision of Orbfit w/r/t  JPL: [0.01494844 0.01706743 0.0120467  ... 0.06763537 0.07834376 0.08087179]


In [12]:
def chart_stats(array):
    median = np.median(array)
    mean = np.mean(array)
    maximum = np.max(array)
    statistics = np.array([median, mean, maximum])
    return median, mean, maximum, statistics

## Graphical Representation

In [13]:
# color coding- fix this
def color_yellow(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

def color_map(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: green']
        elif val < 0.2:
            style = ['background-color: yellow']
        elif val < 0.6:
            style = ['background-color: orange']
        else:
            style = ['background-color: red']
        
        ret += style
    return ret

In [14]:
median1, mean1, max1, statistics1 = chart_stats(great_circle_diff1)
median2, mean2, max2, statistics2 = chart_stats(great_circle_diff2)

total_stats = np.array([statistics1, statistics2]) * u.arcsec

table = pd.DataFrame(data= total_stats.transpose(), columns=['OpenOrb', 'OrbFit'])
table = table.rename(index={0: "Median", 1: "Mean", 2: "Maximum"})
table = table.rename_axis("Metric", axis="columns")
table = table.style.apply(color_map)
table

Metric,OpenOrb,OrbFit
Median,0.024776,0.030403
Mean,0.041732,0.034741
Maximum,0.156815,0.094826


### Key:
value <= 0.05" -> green (good)

value <= 0.2" -> yellow (ok)

value <= 0.6" -> orange (not good but may work)

greater values -> red (bad)

# Prep Zone

Let's adopt these:

value <= 0.05" -> green (good)

value <= 0.2" -> yellow (ok)

value <= 0.6" -> orange (not good but may work)

value is red otherwise (bad)

## Tried it with Ceres and Vesta (MBA population thing)

In [62]:
el_obj = Horizons(id='Ceres', location= '500@10',
               epochs=element_time)
el_jpl = el_obj.elements()
ephem_obj = Horizons(id= 'Ceres', location= Gemini_S,
           epochs={'start': start_time, 'stop':stop_time,
                  'step':'1d'})
ephem_jpl = ephem_obj.ephemerides()

RA_jpl = ephem_jpl['RA']
DEC_jpl = ephem_jpl['DEC']
coord_jpl = np.array([RA_jpl, DEC_jpl]) * u.deg

In [64]:
coord_OpenOrb = get_ephem_OpenOrb(el_jpl)

In [78]:
df = pd.read_fwf('2019OK/1.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55), (56, 61)])

df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
df.loc[df[4] == '-', "DEC"] *= -1

RA_OrbFit = df["RA"]
DEC_OrbFit = df["DEC"]
coord_OrbFit = np.array([RA_OrbFit, DEC_OrbFit]) * u.deg

In [67]:
great_circle_diff1 = max_diff(coord_jpl, coord_OpenOrb)
print("Precision of OpenOrb w/r/t JPL:", great_circle_diff1)

print()
great_circle_diff2 = max_diff(coord_jpl, coord_OrbFit)
print("Precision of Orbfit w/r/t  JPL:", great_circle_diff2)

median1, mean1, max1, statistics1 = chart_stats(great_circle_diff1)
median2, mean2, max2, statistics2 = chart_stats(great_circle_diff2)

total_stats = np.array([statistics1, statistics2]) * u.arcsec

table = pd.DataFrame(data= total_stats.transpose(), columns=['OpenOrb', 'OrbFit'])
table = table.rename(index={0: "Median", 1: "Mean", 2: "Maximum"})
table = table.rename_axis("Metric", axis="columns")
table = table.style.apply(color_map)
table

Precision of OpenOrb w/r/t JPL: [0.01433952 0.01440181 0.01269311 ... 0.04815871 0.04882659 0.02875992]

Precision of Orbfit w/r/t  JPL: [0.01889141 0.03491032 0.02093011 ... 0.07219873 0.07846468 0.05396141]


Metric,OpenOrb,OrbFit
Median,0.020498,0.024051
Mean,0.028702,0.029416
Maximum,0.098048,0.091209


In [77]:
el_obj = Horizons(id='Vesta', location= '500@10',
               epochs=element_time)
el_jpl = el_obj.elements()
ephem_obj = Horizons(id= 'Vesta', location= Gemini_S,
           epochs={'start': start_time, 'stop':stop_time,
                  'step':'1d'})
ephem_jpl = ephem_obj.ephemerides()

RA_jpl = ephem_jpl['RA']
DEC_jpl = ephem_jpl['DEC']
coord_jpl = np.array([RA_jpl, DEC_jpl]) * u.deg

In [80]:
coord_OpenOrb = get_ephem_OpenOrb(el_jpl)

In [90]:
df = pd.read_fwf('2019OK/4.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55), (56, 61)])

df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
df.loc[df[4] == '-', "DEC"] *= -1

RA_OrbFit = df["RA"]
DEC_OrbFit = df["DEC"]
coord_OrbFit = np.array([RA_OrbFit, DEC_OrbFit]) * u.deg

In [93]:
great_circle_diff1 = max_diff(coord_jpl, coord_OpenOrb)
print("Precision of OpenOrb w/r/t JPL:", great_circle_diff1)

print()
great_circle_diff2 = max_diff(coord_jpl, coord_OrbFit)
print("Precision of Orbfit w/r/t  JPL:", great_circle_diff2)

median1, mean1, max1, statistics1 = chart_stats(great_circle_diff1)
median2, mean2, max2, statistics2 = chart_stats(great_circle_diff2)

total_stats = np.array([statistics1, statistics2]) * u.arcsec

table = pd.DataFrame(data= total_stats.transpose(), columns=['OpenOrb', 'OrbFit'])
table = table.rename(index={0: "Median", 1: "Mean", 2: "Maximum"})
table = table.rename_axis("Metric", axis="columns")
table = table.style.apply(color_map)
table

Precision of OpenOrb w/r/t JPL: [0.00307671 0.01084658 0.02322105 ... 0.32576425 0.30639729 0.3183053 ]

Precision of Orbfit w/r/t  JPL: [0.0188552  0.03630032 0.0351147  ... 0.05491089 0.05351543 0.04997519]


Metric,OpenOrb,OrbFit
Median,0.071289,0.040108
Mean,0.102985,0.041699
Maximum,0.478714,0.100057


## Try it with KBO (I picked Eris)

In [70]:
el_obj = Horizons(id='Eris', location= '500@10',
               epochs=element_time)
el_jpl = el_obj.elements()

In [71]:
ephem_obj = Horizons(id= 'Eris', location= Gemini_S,
           epochs={'start': start_time, 'stop':stop_time,
                  'step':'1d'})
ephem_jpl = ephem_obj.ephemerides()

RA_jpl = ephem_jpl['RA']
DEC_jpl = ephem_jpl['DEC']
coord_jpl = np.array([RA_jpl, DEC_jpl]) * u.deg

In [72]:
coord_jpl

<Quantity [[24.31851, 24.31629, 24.31423, ..., 25.58123, 25.57835,
            25.57564],
           [-4.60683, -4.60455, -4.60219, ..., -2.02622, -2.02455,
            -2.02281]] deg>

In [73]:
coord_OpenOrb = get_ephem_OpenOrb(el_jpl)

In [74]:
coord_OpenOrb

<Quantity [[24.3185202 , 24.31629342, 24.31423052, ..., 25.58122954,
            25.57835108, 25.57563611],
           [-4.60683269, -4.60454848, -4.60219202, ..., -2.02621807,
            -2.02455164, -2.022812  ]] deg>

In [75]:
df = pd.read_fwf('2019OK/136199.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55), (56, 61)])

df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
df.loc[df[4] == '-', "DEC"] *= -1

RA_OrbFit = df["RA"]
DEC_OrbFit = df["DEC"]
coord_OrbFit = np.array([RA_OrbFit, DEC_OrbFit]) * u.deg
coord_OrbFit

<Quantity [[24.31852917, 24.3163    , 24.3142375 , ..., 25.58123333,
            25.57835417, 25.5756375 ],
           [-4.60683611, -4.60455278, -4.60219444, ..., -2.026225  ,
            -2.02455833, -2.02281944]] deg>

In [76]:
great_circle_diff1 = max_diff(coord_jpl, coord_OpenOrb)
print("Precision of OpenOrb w/r/t JPL:", great_circle_diff1)

print()
great_circle_diff2 = max_diff(coord_jpl, coord_OrbFit)
print("Precision of Orbfit w/r/t  JPL:", great_circle_diff2)

median1, mean1, max1, statistics1 = chart_stats(great_circle_diff1)
median2, mean2, max2, statistics2 = chart_stats(great_circle_diff2)

total_stats = np.array([statistics1, statistics2]) * u.arcsec

table = pd.DataFrame(data= total_stats.transpose(), columns=['OpenOrb', 'OrbFit'])
table = table.rename(index={0: "Median", 1: "Mean", 2: "Maximum"})
table = table.rename_axis("Metric", axis="columns")
table = table.style.apply(color_map)
table

Precision of OpenOrb w/r/t JPL: [0.03786487 0.01344326 0.00750017 ... 0.00713459 0.00707755 0.01575506]

Precision of Orbfit w/r/t  JPL: [0.07221002 0.03725115 0.03130985 ... 0.02162915 0.03353683 0.03516958]


Metric,OpenOrb,OrbFit
Median,0.014250,0.028000
Mean,0.013678,0.027721
Maximum,0.037865,0.072210


# Recipe

In [15]:
'''
Gfortran version 4.8.6

Useful bits from docs:

JPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'

pyoorb:
orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
perihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART
eccentricity for COM or KEP, y (au) for CART
inclination (deg) for COM or KEP, z (au) for CART
longitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART
argument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT
epoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART
orbital element type; integer value: CART: 1, COM: 2, KEP: 3
epoch of the osculating elements (modified Julian date)
timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
absolute magnitude of the target
photometric slope parameter of the target


JPL TableColumns values to extract:
[0,1,2,3,7,8,23] = [targetname, datetime str, datettime jd, H, RA, DEC, V]

Want pyoorb.oorb_ephemeris_basic , use these indices for properties:
[0,1,2,9] = [mjd, RA (deg), DEC(deg), predicted apparent V-band mag]

example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation
'''

"\nGfortran version 4.8.6\n\nUseful bits from docs:\n\nJPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'\n\npyoorb:\norbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits\nperihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART\neccentricity for COM or KEP, y (au) for CART\ninclination (deg) for COM or KEP, z (au) for CART\nlongitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART\nargument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT\nepoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART\norbital element type; integer value: CART: 1, COM: 2, KEP: 3\nepoch of the osculating elements (modified Julian date)\ntimescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4\nabsolute magnitude of the target\nphotometric slope parameter of the t